<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Brain-tumor-segmentation-using-fastMONAI" data-toc-modified-id="Brain-tumor-segmentation-using-fastMONAI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Brain tumor segmentation using <code>fastMONAI</code></a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Load-and-inspect-the-data" data-toc-modified-id="Load-and-inspect-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load and inspect the data</a></span><ul class="toc-item"><li><span><a href="#Inspect-the-data" data-toc-modified-id="Inspect-the-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Inspect the data</a></span></li><li><span><a href="#Data-augmentation-and-dataloaders" data-toc-modified-id="Data-augmentation-and-dataloaders-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Data augmentation and dataloaders</a></span><ul class="toc-item"><li><span><a href="#What-is-data-augmentation?" data-toc-modified-id="What-is-data-augmentation?-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>What is data augmentation?</a></span></li><li><span><a href="#Create-dataloaders" data-toc-modified-id="Create-dataloaders-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Create dataloaders</a></span></li></ul></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Model-architecture" data-toc-modified-id="Model-architecture-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Model architecture</a></span></li><li><span><a href="#Loss-function" data-toc-modified-id="Loss-function-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Loss function</a></span></li></ul></li><li><span><a href="#Evaluate-results" data-toc-modified-id="Evaluate-results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluate results</a></span><ul class="toc-item"><li><span><a href="#Inference-on-test-data" data-toc-modified-id="Inference-on-test-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Inference on test data</a></span></li></ul></li><li><span><a href="#Export-the-model-and-dataloader" data-toc-modified-id="Export-the-model-and-dataloader-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Export the model and dataloader</a></span></li><li><span><a href="#Extra:-Radiomics" data-toc-modified-id="Extra:-Radiomics-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Extra: Radiomics</a></span></li></ul></div>

# Brain tumor segmentation using `fastMONAI`

This notebook illustrates an approach to constructing a brain tumor segmentation model based on MR images. We aim to extract meaningful tumor regions directly from multimodal MRI (T1w, T1ce, T2w, and FLAIR). In this case, the active tumor (AT), necrotic core (NCR), and peritumoral edematous/infiltrated tissue (ED).

<img width=40% src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/brain_tumor.jpeg">

[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MMIV-ML/fastMONAI/blob/master/presentations/MMIV-1022/MMIV-Oct2022-brain_tumor_segmentation.ipynb)

Here's an illustration of what we want to achieve (illustration taken from the BraTS Challenge):

<img src="https://www.med.upenn.edu/cbica/assets/user-content/images/BraTS/brats-tumor-subregions.jpg">

# Setup

We must first set up the software libraries we'll use to construct our model. Chief among these is the `fastMONAI` library.

<a href="https://fastmonai.no"><img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/fastmonai_no.png"></a>

In [ ]:
#| hide
# This is a quick check of whether the notebook is currently 
# running on Google Colaboratory or on Kaggle, 
# as that makes some difference for the code below.

try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

#Install `fastMONAI` if notebook is running on Google Colab or on Kaggle
if (colab or kaggle):
    %pip install fastMONAI
    if colab:
        from fastMONAI.utils import print_colab_gpu_info
        print_colab_gpu_info()
else:
    print('Running locally')

In [ ]:
from fastMONAI.vision_all import *

from monai.apps import DecathlonDataset
from sklearn.model_selection import train_test_split

# Load and inspect the data

> We will use the brain tumors dataset from the Medical Segmentation Decathlon challenge (http://medicaldecathlon.com/). The data is collected from the Multimodal Brain Tumor Image Segmentation Benchmark Challenge (BraTS) dataset from 2016 and 2017. The task is to segment tumors into three different subregions (active tumor (AT), necrotic core (NCR), and peritumoral edematous/infiltrated tissue (ED)) from multimodal multisite MRI data (T1w, T1ce, T2w, and FLAIR). 

<img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/brats-montage.png">

We use the MONAI function `DecathlonDataset` to download the data and generate items for training. 

In [ ]:
path = Path('data')
path.mkdir(exist_ok=True)

In [ ]:
training_data = DecathlonDataset(root_dir=path, task="Task01_BrainTumour", section="training", download=True,
                                 cache_num=0, num_workers=3)

In [ ]:
df = pd.DataFrame(training_data.data)

We now have a bunch of images and corresponding labels. Here are the first five:

In [ ]:
df.head()

In [ ]:
df.info()

Each of the 388 data sets consists of four 3D volumes (T1, T1c, T2, FLAIR) and corresponding manual labels. Here's one example:<br><br>

<img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/img_182_axial.gif"><img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/img_182_coronal.gif"><img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/img_182_sagittal.gif">

We will train our model on parts of the data set, the so-called **training data**. After training, we'll need some new data to test our model's ability to generalize to unseen data, so-called **test data**. 

This is achieved by splitting the labeled data into two sets: training and test.

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df.shape, test_df.shape

## Inspect the data

`fastMONAI` has a useful function to construct a dataset from a list of labeled images:

In [ ]:
med_dataset = MedDataset(img_list=train_df.label.tolist()[:20], dtype=MedMask, max_workers=12)

It provides useful information about our data set:

In [ ]:
med_dataset.df.head()

In [ ]:
summary_df = med_dataset.summary()

In [ ]:
summary_df

We observe that in this case the voxel spacing is the same for all 349 images, and also that they are oriented identically.

In [ ]:
resample, reorder = med_dataset.suggestion()
resample, reorder

In [ ]:
img_size = med_dataset.get_largest_img_size(resample=resample)
img_size

## Data augmentation and dataloaders

### What is data augmentation?

By doing **data augmentation**, one aims to increase the diversity of a given data set by performing random, realistic transformations of the data. For images, these transformations can be rotations, flips, zooming, pixel intensity modifications, and much more. This also ensures a degree of **invariance** to these transformations for the resulting trained models.

There are many possible data augmentation techniques, ranging from basic to more advanced transformations, including methods for combining multiple images into sets of "new" images (e.g., what's called "CutMix" or "MixUp" and more).

Here are some illustrations of various data augmentation strategies (taken from https://www.quantib.com/blog/image-augmentation-how-to-overcome-small-radiology-datasets): 

<img src="https://www.quantib.com/hs-fs/hubfs/Blog%20and%20news%20images/Examples%20of%20rigid%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib.png?width=1549&name=Examples%20of%20rigid%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib.png">

<img src="https://www.quantib.com/hs-fs/hubfs/assets/images/blog/Examples%20of%20stretch%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib.png?width=3098&name=Examples%20of%20stretch%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib.png">

<img src="https://www.quantib.com/hs-fs/hubfs/Blog%20and%20news%20images/Examples%20of%20elastic%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib.png?width=1448&name=Examples%20of%20elastic%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib.png">

<img src="https://www.quantib.com/hs-fs/hubfs/Blog%20and%20news%20images/Contrast%20shift%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib-1.png?width=1072&name=Contrast%20shift%20augmentation%20-%20AI%20in%20radiology%20-%20Quantib-1.png">

When doing data augmentation, it is vital that

(i) the transformations won't change the correct label (f.ex., zooming in on a region of the image that doesn't contain the information needed to assign the class of the original image. Think zooming in on a part of a bone X-ray that doesn't include the finding of interest, say, a fracture)<br><br>
(ii) be at least somewhat realistic (f.ex., if you expect all the images to have a fixed up-down orientation, as is typically the case in, say, head MRI, vertical flips will not be a good idea).


In our case, we normalize the image, resize them all to the same size, and do some random motion as our data augmentation:

In [ ]:
size=[224,224,128]

In [ ]:
item_tfms = [ZNormalization(), PadOrCrop(size), RandomAffine(scales=0, degrees=5, isotropic=True)] 

After creating dataloaders that apply these transformations, we can have a look at the results.

### Create dataloaders

In [ ]:
dblock = MedDataBlock(blocks=(ImageBlock(cls=MedImage), MedMaskBlock), 
                      splitter=RandomSplitter(seed=42),
                      get_x=ColReader('image'),
                      get_y=ColReader('label'),
                      item_tfms=item_tfms,
                      batch_tfms=None,
                      reorder=reorder,
                      resample=resample) 

In [ ]:
bs=4

In [ ]:
dls = dblock.dataloaders(train_df, bs=bs)

Here's the effect of our data augmentation applied to a single image:

In [ ]:
dls.show_batch(anatomical_plane=0, unique=True)

In [ ]:
# training and validation
len(dls.train_ds.items), len(dls.valid_ds.items)

Here's a batch of data:

In [ ]:
dls.show_batch(anatomical_plane=0) 

# Model

## Model architecture

We use an enhanced version of UNet from MONAI. 

Here's an illustration of the basic UNet architecture on which our model is built:

<img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/unet.jpeg">

In [ ]:
from monai.networks.nets import UNet

## Loss function

The Dice coeffiecient measures the degree of overlap between the predicted tumor mask and the "ground truth" masks:

<img width=20% src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/dice.png">

We use a loss function that combines Dice loss and Cross Entropy loss and returns the weighted sum of these two losses. 

In [ ]:
from monai.losses import DiceCELoss

In [ ]:
codes = np.unique(med_img_reader(train_df.label.tolist()[0]))
n_classes = len(codes)
codes, n_classes

In [ ]:
model = UNet(dimensions=3, in_channels=4, out_channels=n_classes, 
             channels=(16, 32, 64, 128, 256),strides=(2, 2, 2, 2), 
             num_res_units=2)

In [ ]:
loss_func = CustomLoss(loss_func=DiceCELoss(to_onehot_y=True, include_background=True, softmax=True))

## Train the model

Now we're ready to train the model. After training, we'll have something that can produce the following results on new, unseen MR recordings:

<img width=60% src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/prediction_results.png">

<img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/pred_093.gif">

In [ ]:
learn = Learner(dls, model, loss_func=loss_func, opt_func=ranger, metrics=multi_dice_score)#.to_fp16()

In [ ]:
if not (colab or kaggle):
    lr = learn.lr_find()
else:
    lr = 8e-4

In the interest of time, our model is trained for only a few epochs. If you have the time, you can raise this number to something higher (f.ex. `epochs=30` or more) to get a model that performs much better.

In [ ]:
epochs = 1

In [ ]:
learn.fit_flat_cos(epochs, lr)

In [ ]:
if (colab or kaggle):
    !wget https://www.dropbox.com/s/tmebx1m4q57tn7b/trained.braintumor-model.pth?dl=1
    !mkdir models
    !mv trained.braintumor-model.pth?dl=1 models/trained.braintumor-model.pth

In [ ]:
learn.load('trained.braintumor-model')

In [ ]:
#learn.save('trained.braintumor-model')

# Evaluate results

Let's check how the model performs on some validation data:

In [ ]:
learn.show_results(anatomical_plane=0, ds_idx=1)

## Inference on test data

Remember that we also have some unseen test data that we can try our model on:

In [ ]:
test_dl = learn.dls.test_dl(test_df[:10],with_labels=True)

In [ ]:
test_dl.show_batch(anatomical_plane=0, figsize=(10,10))

In [ ]:
pred_acts, labels = learn.get_preds(dl=test_dl)
pred_acts.shape, labels.shape

Dice score for labels 1,2 and 3: 

In [ ]:
multi_dice_score(pred_acts, labels)

In [ ]:
learn.show_results(anatomical_plane=0, dl=test_dl)

# Export the model and dataloader

The final step is to export the model and the pre-processing steps so that they can be used in some further context:

In [ ]:
store_variables(pkl_fn='vars.pkl', var_vals=[reorder, resample])

In [ ]:
learn.export('braintumor_model.pkl')

This model can then, in principle, be taken further into an infrastructure where it can be tested against new data. 

For example, one can use the "research PACS" infrastructure to host and run such models. You've now constructed the "Segmentation application" in the illustration below:

<img width="60%" src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/deploy.png">

# Extra material: Radiomics

Once we've segmented tumors into meaningful subcompartments, we have a set of regions of interest (ROIs) and can start asking many interesting questions. Computing the tumor volumes is an obvious idea. We can also try to compute various shape characteristics. Perhaps the intensity variation in the tumor is a valuable indicator of tumor hetereogeneity. What about the tumor location? 

Extracting features from objects of interest in medical images for diagnostic purposes is often referred to as **radiomics**. The goal of radiomics is to extract information from medical images that can be used as part of a medical imaging-based diagnostic workflow. The information can be extracted from various imaging modalities, e.g., different MRI contrasts, PET imaging, CT imaging, etc. One can then combine it with other sources of information (e.g., demographics, clinical data, genetics). In such a way, radiomics–and radiogenomics–can open the door to sophisticated and powerful analyses.

**Radiomics workflow:**

<img src="https://github.com/MMIV-ML/fastMONAI/raw/master/presentations/MMIV-1022/assets/radiomics.png">

If you're interested, you can have a look at a basic radiomics approach here: https://github.com/MMIV-ML/presimal2022.